In [17]:
import pandas as pd
import plotly.express as px

In [ ]:
fish = pd.read_parquet("data_clean/fish.parquet", index=False, engine='fastparquet')
harbor = pd.read_parquet("data_clean/harbors.parquet", index=False, engine='fastparquet')
locations = pd.read_parquet("data_clean/locations.parquet", index=False, engine='fastparquet')
vessels = pd.read_parquet("data_clean/vessels.parquet", index=False, engine='fastparquet')
deliveries = pd.read_parquet("data_clean/deliveries.parquet", index=False, engine='fastparquet')
ping = pd.read_parquet("data_clean/pings.parquet", index=False, engine='fastparquet')
trans = pd.read_parquet("data_clean/transactions.parquet", index=False, engine='fastparquet')   
fish_locations = pd.read_parquet("data_clean/fish_locatons.parquet", index=False, engine='fastparquet')

In [173]:
fish_locations.to_json("frontend_data/fish_locations.json", orient="records")

In [130]:
vessels[vessels["company"]=="SouthSeafood Express Corp"].head()

,vessel_id,company,flag_country,length_overall,tonnage
67,snappersnatcher7be,SouthSeafood Express Corp,Oceanus,20.0,100.0
202,roachrobberdb6,SouthSeafood Express Corp,Oceanus,130.0,11700.0


# TEST PAPER

In [95]:
selected_port = "City of South Paackland"
selected_date = pd.Timestamp("2035-10-23")

In [9]:
trans

,source,target_harbor,Activities_harbor,fish_species_harbor,kind_harbor,fish_id,date,qty_tons
0,cargo_2035_2394778c,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-11-03,24.375
1,cargo_2035_23956ba0,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-16,18.125
2,cargo_2035_23957cfd,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-20,20.625
3,cargo_2035_23958501,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,gadusnspecificatae4ba,2035-11-07,13.125
4,cargo_2035_23959ab6,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-24,13.125
...,...,...,...,...,...,...,...,...
5302,cargo_2035_3119010f,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-24,10.000
5303,cargo_2035_3119118e,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-25,15.000
5304,cargo_2035_3119237c,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,NaT,NaN
5305,cargo_2035_311936c4,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-30,19.000


In [96]:
# Calcolo esportazioni totali per fish_id nel porto e data selezionati
daily_exports = (
    trans[(trans["target_harbor"] == selected_port) &
          (pd.to_datetime(trans["date"]).dt.date == selected_date.date())]
    .groupby("fish_id", as_index=False)["qty_tons"]
    .sum()
    .rename(columns={"qty_tons": "exports_tons"})
)
id_to_name = dict(zip(fish["id"], fish["entity_name"])) 
daily_exports["fish_name"] = daily_exports["fish_id"].map(id_to_name)


In [97]:
# enrich con nomi pesci
daily_exports = daily_exports.merge(fish, left_on="fish_id", right_on="id", how="left")[["fish_id", "entity_name", "exports_tons"]]

In [98]:
# Le navi arrivate il giorno PRIMA sono quelle che hanno scaricato il pesce
arrival_date = selected_date - pd.Timedelta(days=1)

arrived_vessels = (
    ping[(ping["source"] == selected_port) &
         (ping["time"].dt.date == arrival_date.date())]
    [["target", "time"]]
    .drop_duplicates("target")
)


In [99]:
sea_area = locations.loc[locations["kind"].isin(["Fishing Ground", "Ecological Preserve"])]

In [100]:
for vessel, group in ping.groupby("target"):
    print(f"Vessel: {vessel}, Records: {len(group)}")

Vessel: albacoreangler47d, Records: 1151
Vessel: albacoreassaulter482, Records: 965
Vessel: amberjackassaulterd52, Records: 1265
Vessel: americaneelenthusiastcfa, Records: 1096
Vessel: anchovyassaulterb1c, Records: 2215
Vessel: aquaticangler5c3, Records: 1080
Vessel: aquaticpursuitf31, Records: 1141
Vessel: aquatransit6bc, Records: 27
Vessel: arcticgraylingangler094, Records: 1904
Vessel: athenad34, Records: 742
Vessel: atlanticbluemarlinmarauder0b4, Records: 837
Vessel: atlanticcodcatcherca6, Records: 1412
Vessel: baitedbreath538, Records: 944
Vessel: barracudabaiter8b3, Records: 1157
Vessel: barracudabandit836, Records: 757
Vessel: bassbaiterb9f, Records: 789
Vessel: bassbandit0d5, Records: 1111
Vessel: bigeyetunabanditb73, Records: 669
Vessel: bigeyetunabuccaneera16, Records: 1058
Vessel: blackbullheadbandit801, Records: 722
Vessel: blackdrumbanditc5b, Records: 124
Vessel: bluecatfishcatcher468, Records: 1476
Vessel: bluefintunabandit177, Records: 976
Vessel: bluefishbandit8ec, Reco

In [101]:
def compute_fishing_window(ping):
    """
    Costruisce cicli di pesca tra due 'city' consecutive, indipendentemente dal nome del porto.
    """
    ping = ping.sort_values(["target", "time"])
    records = []

    for vessel, group in ping.groupby("target"):
        group = group.reset_index(drop=True)

        start_idx = None
        start_time = None

        for i, row in group.iterrows():
            # Inizio ciclo (una city)
            if row["kind"] == "city":
                # Se c'è già un ciclo aperto, chiudilo
                if start_idx is not None and i > start_idx + 1:
                    prev_port = group.loc[start_idx]
                    prev_port_end = prev_port["time"] + pd.to_timedelta(prev_port["dwell"], unit="s")
                    next_port_start = row["time"]
                    # Se l'intervallo ha senso (la nave ha viaggiato)
                    if next_port_start > prev_port_end:
                        # Tutti i punti tra i due porti
                        mask = (group["time"] >= prev_port_end) & (group["time"] <= next_port_start)
                        trip = group.loc[mask & (group["kind"] != "city")]

                        if not trip.empty:
                            dwell_summary = (
                                trip.groupby(["source", "kind"], as_index=False)["dwell"].sum()
                            )
                            dwell_summary = dwell_summary.rename(columns={"source": "location_name"})
                            dwell_summary["vessel_name"] = vessel
                            dwell_summary["departure_port"] = prev_port["source"]
                            dwell_summary["arrival_port"] = row["source"]
                            dwell_summary["arrival_time"] = row["time"]

                            # 🧭 Durata finestra: da fine dwell porto A a inizio dwell porto B
                            dwell_summary["window_days"] = max((next_port_start - prev_port_end).days, 1)
                            records.append(dwell_summary)

                # aggiorna ultimo porto
                start_idx = i

    return pd.concat(records, ignore_index=True)


In [102]:
trial_df_fishing_window = compute_fishing_window(ping)

In [103]:
trial_df_fishing_window[trial_df_fishing_window["departure_port"]!= trial_df_fishing_window["arrival_port"]].head(30)

,location_name,kind,dwell,vessel_name,departure_port,arrival_port,arrival_time,window_days
38,Tuna Shelf,Fishing Ground,11960.476614,albacoreangler47d,City of South Paackland,City of Haacklee,2035-04-19 17:01:16.751189,1
45,Tuna Shelf,Fishing Ground,21076.306195,albacoreangler47d,City of Haacklee,City of South Paackland,2035-04-21 02:31:04.647527,1
46,Tuna Shelf,Fishing Ground,5310.708781,albacoreangler47d,City of South Paackland,City of Haacklee,2035-04-23 09:29:18.461919,1
57,Tuna Shelf,Fishing Ground,4976.694947,albacoreangler47d,City of Haacklee,City of South Paackland,2035-04-25 02:39:12.097863,1
61,Nav 3,buoy,6140.721175,albacoreangler47d,City of Haacklee,City of South Paackland,2035-04-28 02:34:34.815884,1
62,Tuna Shelf,Fishing Ground,15443.882829,albacoreangler47d,City of Haacklee,City of South Paackland,2035-04-28 02:34:34.815884,1
63,Tuna Shelf,Fishing Ground,9533.913168,albacoreangler47d,City of South Paackland,City of Haacklee,2035-04-30 10:44:45.137668,1
72,Tuna Shelf,Fishing Ground,5150.254462,albacoreangler47d,City of Haacklee,City of South Paackland,2035-05-02 02:52:05.650991,1
81,Tuna Shelf,Fishing Ground,12795.955761,albacoreangler47d,City of Haacklee,City of South Paackland,2035-05-05 02:31:25.623277,1
82,Tuna Shelf,Fishing Ground,11314.044103,albacoreangler47d,City of South Paackland,City of Haacklee,2035-05-07 11:14:28.503924,1


## ROUTINE VIZ

In [ ]:
# Seleziona una singola nave da visualizzare
selected_vessel = "aquaticangler5c3"

df_vessel = ping[ping["target"] == selected_vessel].copy()

# Ordina cronologicamente e crea intervallo start-end per ciascun punto
df_vessel = df_vessel.sort_values("time")
df_vessel["start"] = df_vessel["time"]
df_vessel["end"] = df_vessel["time"] + pd.to_timedelta(df_vessel["dwell"], unit="s")

# Semplifica: tieni solo i campi rilevanti
df_vessel = df_vessel[["source", "target", "kind", "start", "end"]]

# Colori diversi per tipo di area (porto, fishing ground, preserve, buoy)
color_map = {
    "city": "#1f77b4",
    "Fishing Ground": "#2ca02c",
    "Ecological Preserve": "#ff7f0e",
    "buoy": "#9467bd"
}

fig = px.timeline(
    df_vessel,
    x_start="start",
    x_end="end",
    y="source",
    color="kind",
    color_discrete_map=color_map,
    title=f"Fishing Routine – {selected_vessel}",
    labels={"location_name": "Location", "kind": "Type"}
)

# Inverti l’asse Y per avere i porti in alto
fig.update_yaxes(autorange="reversed")

# Miglioramenti estetici
fig.update_layout(
    height=600,
    xaxis_title="Time",
    yaxis_title="Locations Visited",
    legend_title="Location Type",
    template="simple_white"
)

fig.show()


In [170]:
# Prep data for JSON export
df_for_routine = ping.sort_values("time")
df_for_routine["start"] = df_for_routine["time"]
df_for_routine["end"] = df_for_routine["time"] + pd.to_timedelta(df_for_routine["dwell"], unit="s")

# Semplifica: tieni solo i campi rilevanti
df_for_routine = df_for_routine[["source", "target", "kind", "start", "end"]]
df_for_routine.to_json("frontend_data/ping_for_timeline.json", orient="records", date_format="iso")



In [105]:
test_date = pd.Timestamp("2035-11-04") 
trial_df_fishing_window[
    (trial_df_fishing_window["vessel_name"] == selected_vessel) &
    (trial_df_fishing_window["location_name"]=="Ghoti Preserve") 
]


,location_name,kind,dwell,vessel_name,departure_port,arrival_port,arrival_time,window_days


## CARGO ESTIM.

In [137]:
selected_port = "City of Lomark"
selected_date = pd.Timestamp("2035-02-21")

# 1️⃣ Calcolo routine di pesca dinamica (tra porti)
dwell_dynamic = compute_fishing_window(ping)

# 2️⃣ Seleziono solo le navi che arrivano nel porto scelto nel giorno prima dell’export
arrival_date = selected_date - pd.Timedelta(days=1)

dwell_dynamic = dwell_dynamic[
    (dwell_dynamic["arrival_port"] == selected_port) &
    (dwell_dynamic["arrival_time"].dt.date == arrival_date.date())
]

# 3️⃣ Aggiungo stazza (tonnage)
vessel_info = vessels[["vessel_id", "tonnage"]].rename(columns={"vessel_id": "vessel_name"})
dwell_dynamic = dwell_dynamic.merge(
    vessel_info,
    on="vessel_name",
    how="left"
)

# 4️⃣ Calcolo esportazioni totali per pesce nel porto e data selezionati
daily_exports = (
    trans[
        (trans["target_harbor"] == selected_port) &
        (pd.to_datetime(trans["date"]).dt.date == selected_date.date())
    ]
    .groupby("fish_id", as_index=False)["qty_tons"]
    .sum()
    .rename(columns={"qty_tons": "exports_tons"})
)

# 5️⃣ Aggiungo nomi dei pesci
id_to_name = dict(zip(fish["id"], fish["entity_name"]))
daily_exports["fish_name"] = daily_exports["fish_id"].map(id_to_name)

# 6️⃣ Costruisco daily view
daily_view = {
    "date": selected_date,
    "port": selected_port,
    "exports": daily_exports,        # esportazioni del giorno
    "vessels_dwell": dwell_dynamic,  # attività navi arrivate il giorno prima
}

print("\n✅ Daily Harbor View (dynamic window) ready!")
print(f"Navi arrivate a {selected_port} il {arrival_date.date()}: {dwell_dynamic['vessel_name'].nunique()}")



✅ Daily Harbor View (dynamic window) ready!
Navi arrivate a City of Lomark il 2035-02-20: 16


In [163]:
print(min(trans["date"]))

2035-02-04 00:00:00


In [152]:
daily_exports = (
    trans[
        (trans["target_harbor"] == selected_port) &
        (pd.to_datetime(trans["date"]).dt.date == selected_date.date())
    ]
    .groupby("fish_id", as_index=False)["qty_tons"]
    .sum()
    .rename(columns={"qty_tons": "exports_tons"})
)


In [162]:
dates = sorted([i for i in trans[
        (trans["target_harbor"] == selected_port) 
        #(pd.to_datetime(trans["date"]).dt.date == selected_date.date())
    ]["date"].unique()
])
dates

[Timestamp('2035-06-07 00:00:00'),
 Timestamp('2035-06-10 00:00:00'),
 Timestamp('2035-06-14 00:00:00'),
 Timestamp('2035-06-17 00:00:00'),
 Timestamp('2035-06-24 00:00:00'),
 Timestamp('2035-06-28 00:00:00'),
 Timestamp('2035-07-01 00:00:00'),
 Timestamp('2035-07-05 00:00:00'),
 Timestamp('2035-07-08 00:00:00'),
 Timestamp('2035-07-12 00:00:00'),
 Timestamp('2035-07-15 00:00:00'),
 Timestamp('2035-07-19 00:00:00'),
 Timestamp('2035-07-22 00:00:00'),
 Timestamp('2035-07-26 00:00:00'),
 Timestamp('2035-07-29 00:00:00'),
 Timestamp('2035-08-01 00:00:00'),
 Timestamp('2035-08-02 00:00:00'),
 Timestamp('2035-08-05 00:00:00'),
 Timestamp('2035-08-06 00:00:00'),
 Timestamp('2035-08-09 00:00:00'),
 Timestamp('2035-08-12 00:00:00'),
 Timestamp('2035-08-16 00:00:00'),
 Timestamp('2035-08-18 00:00:00'),
 Timestamp('2035-08-19 00:00:00'),
 Timestamp('2035-08-23 00:00:00'),
 Timestamp('2035-08-25 00:00:00'),
 Timestamp('2035-08-26 00:00:00'),
 Timestamp('2035-08-30 00:00:00'),
 Timestamp('2035-09-

In [167]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5307 entries, 0 to 5306
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   source               5307 non-null   object        
 1   target_harbor        5307 non-null   object        
 2   Activities_harbor    5307 non-null   object        
 3   fish_species_harbor  5307 non-null   object        
 4   kind_harbor          5307 non-null   object        
 5   fish_id              5307 non-null   object        
 6   date                 4992 non-null   datetime64[ns]
 7   qty_tons             4992 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 331.8+ KB


In [149]:
dwell_dynamic

,location_name,kind,dwell,vessel_name,departure_port,arrival_port,arrival_time,window_days,tonnage
0,Cod Table,Fishing Ground,185660.656461,bigeyetunabuccaneera16,City of Lomark,City of Lomark,2035-02-20 10:10:06.726770,2,300.0
1,Exit East,buoy,25835.208091,bigeyetunabuccaneera16,City of Lomark,City of Lomark,2035-02-20 10:10:06.726770,2,300.0
2,Nav 1,buoy,9817.055264,bigeyetunabuccaneera16,City of Lomark,City of Lomark,2035-02-20 10:10:06.726770,2,300.0
3,Nav 2,buoy,10729.375629,bigeyetunabuccaneera16,City of Lomark,City of Lomark,2035-02-20 10:10:06.726770,2,300.0
4,Nav C,buoy,8622.842932,bigeyetunabuccaneera16,City of Lomark,City of Lomark,2035-02-20 10:10:06.726770,2,300.0
...,...,...,...,...,...,...,...,...,...
78,Cod Table,Fishing Ground,370640.288305,whitesuckerwrangler0b3,City of Lomark,City of Lomark,2035-02-20 04:26:12.575366,7,600.0
79,Exit East,buoy,183926.584010,whitesuckerwrangler0b3,City of Lomark,City of Lomark,2035-02-20 04:26:12.575366,7,600.0
80,Nav 1,buoy,15910.586115,whitesuckerwrangler0b3,City of Lomark,City of Lomark,2035-02-20 04:26:12.575366,7,600.0
81,Nav 2,buoy,31420.341983,whitesuckerwrangler0b3,City of Lomark,City of Lomark,2035-02-20 04:26:12.575366,7,600.0


In [139]:
fish_to_locations = (
        fish_locations.groupby("entity_name")["location_id"]
        .apply(list)
        .to_dict()
)

In [140]:
fish_to_locations

{'Beauvoir/Habeas pisces': ['Cod Table',
  'Ghoti Preserve',
  'Wrasse Beds',
  'Nemo Reef',
  'Don Limpet Preserve',
  'Tuna Shelf'],
 'Birdseye/Pisces frigus': ['Cod Table',
  'Wrasse Beds',
  'Nemo Reef',
  'Don Limpet Preserve',
  'Tuna Shelf'],
 'Cod/Gadus n.specificatae': ['Cod Table'],
 'Harland/Piscis sapidum': ['Tuna Shelf'],
 'Helenaa/Pisces satis': ['Ghoti Preserve',
  'Nemo Reef',
  'Don Limpet Preserve'],
 'Offidiaa/Piscis osseus': ['Ghoti Preserve'],
 'Sockfish/Pisces foetida': ['Don Limpet Preserve'],
 'Tuna/Thunnini n.vera': ['Nemo Reef', 'Don Limpet Preserve', 'Tuna Shelf'],
 'Wrasse/Labridae n.refert': ['Ghoti Preserve', 'Wrasse Beds', 'Nemo Reef']}

In [141]:
def estimate_vessel_catch_by_habitat(daily_exports, dwell_dynamic, fish_locations):
    """
    Associa le esportazioni giornaliere alle navi candidate in base alle aree di pesca (habitat),
    pesando per tonnage × dwell solo nelle location associate al pesce.
    """
    results = []

    if daily_exports.empty or dwell_dynamic.empty:
        return pd.DataFrame(columns=[
            "vessel_name", "fish_name", "location_name", "dwell", "tonnage", "estimated_tons"
        ])

    # Crea una mappa fish → lista di location dove vive
    fish_to_locations = (
        fish_locations.groupby("entity_name")["location_id"]
        .apply(list)
        .to_dict()
    )

    for _, row in daily_exports.iterrows():
        fish_name = row["fish_name"]
        exports_tons = row["exports_tons"]
        print(fish_name)
        habitats = fish_to_locations.get(fish_name, [])
        if not habitats:
            continue
        print(habitats)
        # Filtra solo dwell in location compatibili
        candidates = dwell_dynamic[dwell_dynamic["location_name"].isin(habitats)].copy()
        if candidates.empty:
            continue
        print(candidates)
        candidates["weight"] = candidates["tonnage"] * candidates["dwell"]
        total_weight = candidates["weight"].sum()
        if total_weight == 0:
            continue

        candidates["estimated_tons"] = exports_tons * (candidates["weight"] / total_weight)
        candidates["fish_name"] = fish_name

        results.append(
            candidates[["vessel_name", "fish_name", "location_name", "dwell", "tonnage", "estimated_tons"]]
        )

    if results:
        return pd.concat(results, ignore_index=True)
    else:
        return pd.DataFrame(columns=[
            "vessel_name", "fish_name", "location_name", "dwell", "tonnage", "estimated_tons"
        ])


In [147]:
vessel_catch = estimate_vessel_catch_by_habitat(
    daily_exports,
    dwell_dynamic,
    fish_locations
)

In [146]:
vessel_catch

,vessel_name,fish_name,location_name,dwell,tonnage,estimated_tons


In [124]:
daily_exports.sort_values(by="exports_tons", ascending=False)

,fish_id,exports_tons,fish_name
2,piscesfoetidaae7,66.625,Sockfish/Pisces foetida
0,gadusnspecificatae4ba,28.875,Cod/Gadus n.specificatae
1,habeaspisces4eb,25.500,Beauvoir/Habeas pisces
3,piscesfrigus900,4.125,Birdseye/Pisces frigus


In [144]:
vessel_catch.sort_values(by="estimated_tons", ascending=False)

,vessel_name,fish_name,location_name,dwell,tonnage,estimated_tons


In [126]:
vessels["vessel_id"].unique()

array(['wavewranglerc2d', 'yellowfintunataker08b',
       'arcticgraylingangler094', 'anchovyassaulterb1c', 'oceanreaper44a',
       'herringharpooner843', 'halibuthunterd84',
       'brooktroutbuccaneerc0b', 'groupergrabber60d',
       'welscatfishwrangler6ae', 'skipjacktunatakerf85',
       'seawaysavvy9847', 'pacificcodcapturer81c', 'bluefishbandit8ec',
       'maritimemaelstromc5d', 'europeanperchpirated9b',
       'whitefishwrangler7df', 'longfintunalooterf32', 'plaicepirate8c6',
       'pikeplunderer0ca', 'albacoreangler47d', 'pikepirate89a',
       'flatheadcatfishfindera51', 'burbotbandit7bf', 'clamclaimer13d',
       'neptunesnete8c', 'shrimpshooter91c', 'aquaticangler5c3',
       'blackbullheadbandit801', 'turbottakerd86', 'seaswindler255',
       'atlanticbluemarlinmarauder0b4', 'laketroutlooter534',
       'dolphindasher004', 'oceanicoracle9da', 'tidetamerbe1',
       'mackerelmarauder2d4', 'grasspickerelgangster7d1',
       'harborharmonyfc9', 'barracudabaiter8b3', 'nautic

In [143]:
vessel_catch.sort_values(by="estimated_tons", ascending=False)

,vessel_name,fish_name,location_name,dwell,tonnage,estimated_tons


In [133]:
illegal_fishes = ["Sockfish/Pisces foetida", "Offidiaa/Piscis osseus", "Helenaa/Pisces satis"]
vessels_illegal = vessel_catch[vessel_catch["fish_name"].isin(illegal_fishes)]
vessels_illegal

,vessel_name,fish_name,location_name,dwell,tonnage,estimated_tons
